In [11]:
import numpy as np
import torch
import torch.optim as optim

import cv2
import scipy.ndimage
import os

import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
from torchvision import transforms
import torch.utils.data as data

from timeit import default_timer as timer
import random
from random import shuffle

import matplotlib.pyplot as plt  
%matplotlib inline

import import_ipynb
import data_augmentation
import scipy.ndimage

from PIL import Image

In [12]:
def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        if filename == '.DS_Store':
          print(filename)
        else:
          img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images[filename.split('.')[0]] = img  #remove extension from file name
    return images

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [14]:
train_data_1 = load_images_from_folder("../data/Training-Dataset/Resize-Augmented-Masks/1/")
train_data_2 = load_images_from_folder("../data/Training-Dataset/Resize-Augmented-Masks/2/")
train_data_3 = load_images_from_folder("../data/Training-Dataset/Resize-Augmented-Masks/3/")
train_data_4 = load_images_from_folder("../data/Training-Dataset/Resize-Augmented-Masks/4/")
train_data_5 = load_images_from_folder("../data/Training-Dataset/Resize-Augmented-Masks/5/")

.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store


In [15]:
test_data_1 = load_images_from_folder("../data/Validation-Dataset/Resize-Augmented-Predicted-Masks/1/")
test_data_2 = load_images_from_folder("../data/Validation-Dataset/Resize-Augmented-Predicted-Masks/2/")
test_data_3 = load_images_from_folder("../data/Validation-Dataset/Resize-Augmented-Predicted-Masks/3/")
test_data_4 = load_images_from_folder("../data/Validation-Dataset/Resize-Augmented-Predicted-Masks/4/")
test_data_5 = load_images_from_folder("../data/Validation-Dataset/Resize-Augmented-Predicted-Masks/5/")

.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store


In [16]:
def labeledImages(images): 
    return [int(key[0])-1 for key in images.keys()]

In [ ]:
aug1, lab1 = data_augmentation.extensiveDataAugmentation(train_data_1, 1)
aug2, lab2 = data_augmentation.extensiveDataAugmentation(train_data_2, 2)
aug3, lab3 = data_augmentation.extensiveDataAugmentation(train_data_3, 3)
aug4, lab4 = data_augmentation.extensiveDataAugmentation(train_data_4, 4)
aug5, lab5 = data_augmentation.extensiveDataAugmentation(train_data_5, 5)

augmenting
resizing
augmenting


In [ ]:
aug1_val, lab1_val = data_augmentation.extensiveDataAugmentation(test_data_1, 1, augment=False)
aug2_val, lab2_val = data_augmentation.extensiveDataAugmentation(test_data_2, 2, augment=False)
aug3_val, lab3_val = data_augmentation.extensiveDataAugmentation(test_data_3, 3, augment=False)
aug4_val, lab4_val = data_augmentation.extensiveDataAugmentation(test_data_4, 4, augment=False)
aug5_val, lab5_val = data_augmentation.extensiveDataAugmentation(test_data_5, 5, augment=False)

In [ ]:
aug1_val_id, lab1_val_id = data_augmentation.extensiveDataAugmentation(test_data_1_id, 1, augment=False)
aug2_val_id, lab2_val_id = data_augmentation.extensiveDataAugmentation(test_data_2_id, 2, augment=False)
aug3_val_id, lab3_val_id = data_augmentation.extensiveDataAugmentation(test_data_3_id, 3, augment=False)
aug4_val_id, lab4_val_id = data_augmentation.extensiveDataAugmentation(test_data_4_id, 4, augment=False)
aug5_val_id, lab5_val_id = data_augmentation.extensiveDataAugmentation(test_data_5_id, 5, augment=False)

In [ ]:
aug = np.concatenate((aug1,aug2,aug3,aug4,aug5))
labels = np.concatenate((lab1,lab2,lab3,lab4,lab5))

In [ ]:
aug_val = np.concatenate((aug1_val,aug2_val,aug3_val,aug4_val,aug5_val))
labels_val = np.concatenate((lab1_val,lab2_val,lab3_val,lab4_val,lab5_val))

In [ ]:
aug_val_id = np.concatenate((aug1_val_id,aug2_val_id,aug3_val_id,aug4_val_id,aug5_val_id))
labels_val_id = np.concatenate((lab1_val_id,lab2_val_id,lab3_val_id,lab4_val_id,lab5_val_id))

In [ ]:
np.unique(aug1[4,:,:,0])

In [ ]:
aug[0,:,:,0] == 1

In [ ]:
plt.imshow(aug1[120,:,:,0])

In [ ]:
tensor_x = torch.FloatTensor(aug_train)
tensor_y = torch.LongTensor(labels_train)

my_dataset = data.TensorDataset(tensor_x.permute(0,3,1,2),tensor_y) 
trainloader = data.DataLoader(my_dataset, batch_size=60, shuffle = True)


In [ ]:
tensor_x_val = torch.FloatTensor(aug_test)
tensor_y_val = torch.LongTensor(labels_test)

my_dataset_val = data.TensorDataset(tensor_x_val.permute(0,3,1,2),tensor_y_val) 
validloader = data.DataLoader(my_dataset_val, batch_size=60, shuffle = True) 


In [ ]:
tensor_x_val_id = torch.FloatTensor(aug_val_id)
tensor_y_val_id = torch.LongTensor(labels_val_id)

my_dataset_val_id = data.TensorDataset(tensor_x_val_id.permute(0,3,1,2),tensor_y_val_id) 
validloader_id = data.DataLoader(my_dataset_val_id, batch_size=60, shuffle = True) 

In [ ]:
iter_ = iter(trainloader)
bimg, blabel = next(iter_)

print('Batch Label shape: ', blabel.shape)
print('The Batched tensors return a collection of {} RGB images ({} channel, {} height pixels, {} width pixels)'.format(bimg.shape[0],
                                                                                                                        bimg.shape[1],
                                                                                                                       bimg.shape[2],
                                                                                                                        bimg.shape[3]))
print('In the case of the labels, we obtain {} batched integers, one per image'.format(blabel.shape[0]))

In [ ]:
class BigNet(nn.Module):
  
  def __init__(self):
    super().__init__()
    
    self.conv1 = nn.Conv2d(1, 4 , 3, 1, padding=1)
    self.conv_bn_1 = nn.BatchNorm2d(4)
    self.conv2 = nn.Conv2d(4, 8, 3, 1, padding=1)
    self.conv_bn_2 = nn.BatchNorm2d(8)    
    self.conv3 = nn.Conv2d(8, 16 , 3, 1, padding=1)
    self.conv_bn_3 = nn.BatchNorm2d(16)
    self.conv4 = nn.Conv2d(16, 32 , 3, 1, padding=1)
    self.conv_bn_4 = nn.BatchNorm2d(32)

    self.pool  = nn.MaxPool2d(2,2)
    self.act   = nn.ReLU(inplace=False)
    self.drop = nn.Dropout2d(0.1)    
    
    self.mlp = nn.Sequential(
        nn.Linear(16 * 16 * 32, 128),
        nn.ReLU(),
        nn.Dropout(0.8),
        nn.Linear(128,5),        
        nn.Softmax(dim=-1)    
    )
 
    
  def forward(self, x):
    x = self.conv_bn_1(self.conv1(x))
    x = self.pool(self.act(x))
    x = self.drop(x)

    x = self.conv_bn_2(self.conv2(x))
    x = self.pool(self.act(x))
    x = self.drop(x)

    x = self.conv_bn_3(self.conv3(x))
    x = self.pool(self.act(x))
    x = self.drop(x)

    x = self.conv_bn_4(self.conv4(x))
    x = self.pool(self.act(x))
    x = self.drop(x)

    bsz, nch, height, width = x.shape
    x = x.view(bsz, -1)
    
    y = self.mlp(x)

    return y

In [ ]:
def model_params(model):
    # from: https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/6
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
cnn = BigNet()
print(model_params(cnn))
cnn.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.1, momentum = 0.2)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 25, gamma = 0.1)
epochs = 50
steps = 0
running_loss = 0
train_losses = []
test_losses = []
print_every = 3

In [ ]:
for epoch in range(epochs):
    for inputs, labels in trainloader:
        accuracy_train = 0
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs[:,0,:,:]
        inputs = inputs.unsqueeze(dim=1)
        optimizer.zero_grad()
        logps = cnn.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        batch_loss = criterion(logps, labels)
        test_loss += batch_loss.item()
                    
        #ps = torch.exp(logps)
        top_p, top_class = logps.topk(1, dim=1)
        #print(top_class[1:10], logps[1:10])
        equals = top_class == labels.view(*top_class.shape)
        accuracy_train += torch.mean(equals.type(torch.FloatTensor)).item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            cnn.eval()
            with torch.no_grad():
                for inputs_val, labels_val in validloader:
                    #print(labels[0])
                    inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)
                    
                    inputs_val = inputs_val[:,0,:,:]
                    inputs_val = inputs_val.unsqueeze(dim=1)
                    
                    logps = cnn.forward(inputs_val)

                    batch_loss = criterion(logps, labels_val)
                    test_loss += batch_loss.item()
                    
                    #ps = torch.exp(logps)
                    top_p, top_class = logps.topk(1, dim=1)
                    #print(top_class[1:10], logps[1:10])
                    equals = top_class == labels_val.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(validloader))                    
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Train accuracy:{accuracy_train:.3f}.. "
                  f"Test loss: {test_loss/len(validloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(validloader):.3f}")
                
            running_loss = 0
            cnn.train()


torch.save(cnn, 'cnn_finger.pth')

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()